In [1]:
import pandas as pd
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290
...,...,...,...,...
9995,97112.86,0,12,25060.64248
9996,107424.63,1,16,21317.31764
9997,93856.99,0,4,20950.38812
9998,51250.57,1,7,16840.13376


In [2]:
# Separando dados de treino e teste 75% treino 25% teste
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']
SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify= y)
print('Treinaremos com {} elementos e testaremos com {} elementos.'.format(len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos.


In [3]:
# Definindo o que é uma taxa de acerto aceitável por meio de uma baseline
from sklearn.dummy import DummyClassifier
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100
print('A eficácia do dummy foi de {:.2f}%' .format(acuracia))


A eficácia do dummy foi de 58.00%


In [4]:
# Gerando uma árvore de decisão
from sklearn.tree import DecisionTreeClassifier
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth = 2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('Taxa de acerto: {:.2f}%' .format(acuracia))

'''
Perceba que se mudarmos o SEED mudamos a taxa de acerto (para mais ou para menos)
e a validação de modelos não devem depender dessa aleatoriedade. É preciso utilizar
recursos mais sofisticados para dizer se um modelo é válido ou não

'''


Taxa de acerto: 71.92%


'\nPerceba que se mudarmos o SEED mudamos a taxa de acerto (para mais ou para menos)\ne a validação de modelos não devem depender dessa aleatoriedade. É preciso utilizar\nrecursos mais sofisticados para dizer se um modelo é válido ou não\n\n'

In [5]:
# Treino - Teste (Holdout)
''' Podemos ser afetados pela aleatoriedade no treino e teste e obter resultados não tão confiáveis.
No caso de separar uma amostra pra teste e der 35% de acerto, e com outra dar 85%, faremos a média dos resultados
e temos o resultado final, no caso 60%.
O método utilizado será o Cross validation, separamos em n partes e cruzamos esses testes e treinos. É o K-fold (k pedaços)

 '''

from sklearn.model_selection import cross_validate
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth = 2)
resultados = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = resultados['test_score'].mean()
desvio_padrao = resultados['test_score'].std()
print('Accuracy [{:.2f}, {:.2f}]' .format((media - 2 * desvio_padrao) * 100, (media +  2 * desvio_padrao) * 100)) # Intervalo de confiança de 95%

Accuracy [74.99, 76.57]


In [6]:
# Com 10 divisões
from sklearn.model_selection import cross_validate
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth = 10)
resultados = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = resultados['test_score'].mean()
desvio_padrao = resultados['test_score'].std()
print('Accuracy [{:.2f}, {:.2f}]' .format((media - 2 * desvio_padrao) * 100, (media +  2 * desvio_padrao) * 100)) # Intervalo de confiança de 95%

Accuracy [75.96, 77.42]


In [7]:
# Com 5 divisões (divisões entre 5 e 10 jã são suficientes)
from sklearn.model_selection import cross_validate
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth = 5)
resultados = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = resultados['test_score'].mean()
desvio_padrao = resultados['test_score'].std()
print('Accuracy [{:.2f}, {:.2f}]' .format((media - 2 * desvio_padrao) * 100, (media +  2 * desvio_padrao) * 100)) # Intervalo de confiança de 95%

Accuracy [77.96, 79.12]


In [8]:
def imprime_resultados(resultados):
    media = resultados['test_score'].mean()
    desvio_padrao = resultados['test_score'].std()
    print('Accuracy [{:.2f}, {:.2f}]' .format((media - 2 * desvio_padrao) * 100, (media +  2 * desvio_padrao) * 100)) # Intervalo de confiança de 95%

In [9]:
#Aletoriedade no cross validation
from sklearn.model_selection import KFold
SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(resultados)

Accuracy [74.37, 77.19]


In [10]:
#Com Shuffle
from sklearn.model_selection import KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle= True)
modelo = DecisionTreeClassifier(max_depth = 2)
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(resultados)

Accuracy [73.26, 78.26]


In [11]:
#  Cross validation com Standard Scaler
from sklearn.preprocessing import StandardScaler
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado =  scaler.transform(teste_x)

from sklearn.svm import SVC



modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('Taxa de acerto: {:.2f}%' .format(acuracia))

Taxa de acerto: 74.40%


In [12]:

from sklearn.model_selection import GroupKFold
SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits = 10)
modelo = SVC()
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(resultados)

Accuracy [74.76, 79.54]


In [13]:
from sklearn.pipeline import Pipeline
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

Pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = KFold(n_splits = 10)
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(resultados)

Accuracy [74.76, 79.54]
